### EXAMEN NLP 

### IA SCHOOL : 2020-2021

| Prénoms       |     Nom         |   
| ------------- |: -------------: |
| Amadou lamarana      | DIALLO               |


<hr>

# ETPAE 3: CLASSIFIEURS BINAIRES

### Nous allons dans cette partie mettre en plca des classifieurs binaires sur les données d'entraiement et de test issues des 3 méthodes précédentes..
* TF-IDF
* Word2Vec
* CAMEMBERT

ETAPE 3: MISE EN PLACE DES MODÈLES DE CLASSIFICATION

In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import feature_extraction, model_selection, pipeline,metrics,  manifold, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
import pandas as pd

## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
##Word2Vec
from gensim.models import Word2Vec
### CAMEMBERT
import torch
from transformers import CamembertTokenizer,CamembertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

#### camembert
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

#### Dataset issu du scraping sur Amazon

In [2]:
commentaires_raw = pd.read_csv('commentaires_nettoyes_cinq_blessures_ok.csv')
commentaires_raw = commentaires_raw.sample(frac=1).reset_index(drop=True)

In [3]:
y = commentaires_raw['label'].tolist()
X = commentaires_raw['commentaire'].tolist()

### Méthode utilisant TF_IDF

In [4]:
## pipeline
vectorizer = TfidfVectorizer(ngram_range=(1, 4))
X_tfidf = vectorizer.fit_transform(commentaires_raw['commentaire'])

## split des données
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.4, random_state=43)


In [ ]:
Mat=pd.DataFrame(X_tfidf.toarray(),columns=vectorizer.get_feature_names())
Mat

**CLASSIFIEUR NAIVE BAYES**

 ****Nous notons une accuracy de 60%  et une fort taux de faux négatif****

In [6]:
clf_nb = MultinomialNB()
## train classifier
clf_nb.fit(X_train_tfidf, y_train)
preds = clf_nb.predict(X_test_tfidf)
print(accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

0.74
[[64 36]
 [16 84]]
              precision    recall  f1-score   support

           0       0.80      0.64      0.71       100
           1       0.70      0.84      0.76       100

    accuracy                           0.74       200
   macro avg       0.75      0.74      0.74       200
weighted avg       0.75      0.74      0.74       200



#### REGRESSION LOGISTIQUE

****Nous notons une accuracy de 66%  et une fort taux de faux négatif****

In [7]:
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)
lr.score(X_test_tfidf, y_test)
preds = lr.predict(X_test_tfidf)
print(accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

0.78
[[75 25]
 [19 81]]
              precision    recall  f1-score   support

           0       0.80      0.75      0.77       100
           1       0.76      0.81      0.79       100

    accuracy                           0.78       200
   macro avg       0.78      0.78      0.78       200
weighted avg       0.78      0.78      0.78       200



**Les méthodes classiques de classification ne semblent pas donner d'excellents résultats**

### METHODE  WORD2VEC

##### Split des vecteurs

### Nous chargeons notre modèle déja implémenté à la précédente étape

In [8]:
model_wv = Word2Vec.load("word2vec.model")

In [ ]:
model_wv.corpus_count

In [10]:
commentaires_raw= pd.read_csv('commentaires_nettoyes_cinq_blessures_ok.csv')

In [11]:
X_train_wv, X_test_wv, y_train_wv, y_test_wv = train_test_split(commentaires_raw[['commentaire']], commentaires_raw['label'], test_size=0.2, random_state=43)

#### On fait la somme des vecteurs associés aux mots qui la composent ou pas si le mot n’est pas dans le vocabulaire.

In [ ]:
import numpy

def recuperer_vecteur(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def somme_vecteurs(phrase, model):
    return sum(recuperer_vecteur(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([somme_vecteurs(p, model) for p in X])
    return feats

wv_train_feat = word2vec_features(X_train_wv['commentaire'], model_wv)

wv_train_feat.shape

pipe = make_pipeline(TfidfVectorizer(), TfidfTransformer())
pipe.fit(X_train_wv['commentaire'])
feat_train = pipe.transform(X_train_wv)
feat_train.shape


**Entrainement du modèle**

In [ ]:
clfwv = LogisticRegression(solver='lbfgs', max_iter=1000)
clfwv.fit(wv_train_feat, y_train_wv)

**Données de test**

In [14]:
wv_test_feat = word2vec_features(X_test_wv["commentaire"], model_wv)

**Evaluation du modèle**

In [15]:
wv_test_feat = word2vec_features(X_test_wv["commentaire"], model_wv)

In [ ]:
#On peut constater que la fonction de similarités ne retourne pas des résultat très intéressants.

**Un mauvais score 47%  avec le model word2Vec, il peut être améliorer avec une réduction des composantes**

In [16]:
y_pred = clfwv.predict(wv_test_feat)
print(accuracy_score(y_test_wv, y_pred))
print(confusion_matrix(y_test_wv, y_pred))
print(classification_report(y_test_wv, y_pred))

0.47
[[ 0 53]
 [ 0 47]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        53
           1       0.47      1.00      0.64        47

    accuracy                           0.47       100
   macro avg       0.23      0.50      0.32       100
weighted avg       0.22      0.47      0.30       100



#### Réduction de la dimension 

In [17]:
from sklearn.decomposition import TruncatedSVD
pipe_svd = make_pipeline(CountVectorizer(), TruncatedSVD(n_components=300))
pipe_svd.fit(X_train_wv['commentaire'])
feat_train_svd = pipe_svd.transform(X_train_wv['commentaire'])
feat_test_svd = pipe_svd.transform(X_test_wv['commentaire'])
feat_train_svd.shape

(400, 300)

In [18]:
clf_svd = RandomForestClassifier(n_estimators=200)
clf_svd.fit(feat_train_svd, y_train_wv)
y_pred = clf_svd.predict(feat_test_svd)


In [19]:
#feat_test_svd = pipe_svd.transform(X_test_wv['commentaire'])
clf_svd.score(feat_test_svd, y_test_wv)

0.65

**La performance améliore avec 60% mais faiblement par rapport  à la méthode sans l'ACP. Il faudrait sans doute jouer avec les hyperparamètres de l’apprentissage ou réutiliser un model appris sur un corpus similaire aux données initiales mais nettement plus grand. On peut constater que la fonction de similarités ne retourne pas des résultat très intéressants**

### MODELE AVEC CAMEMBERT

Ensuite nous allons importer le modèle, le tokenizer et les weights pré-entraînés. Les weights sont comme les word embeddings. Deux choses à noter :

Bert a son propre tokenizer avec un vocabulaire fixe. Il est donc inutile de tokénisez vous-même.

Ces weights sont issus du modèle à l’état “brut”.

En pratique vous devez fine-tuner Camembert sur des données à vous afin d’avoir des weights propres à chaque tâche et corpus.

In [ ]:
# load model, tokenizer and weights
camembert, tokenizer, weights = (ppb.CamembertModel, ppb.CamembertTokenizer, 'camembert/camembert-large')

# Load pretrained model/tokenizer
tokenizer = tokenizer.from_pretrained(weights)
model = camembert.from_pretrained(weights)

In [21]:
comments = commentaires_raw['commentaire']

Bert ne sait que tokéniser des phrases de longueur maximale de 512 tokens. Ici nous allons simplement enlever les commentaires trop longs.

#### Le commentaire le plus long est de: 115 tokens

In [ ]:
# see if there are length > 512
max_len = 0
for i,sent in enumerate(comments):
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    if len(input_ids) > 512:
        print("annoying review at", i,"with length",
              len(input_ids))
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Maintenant que la phrase la plus longue enlevée, nous faisons un padding de 115 tokens pour homogénéiser la longueur de phrases. Cela rendra l’entraînement plus simple. Nous indiquons aussi où se trouve les paddings avec np.where pour que Bert sache traiter les tokens de padding.

In [ ]:
tokenized = comments.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

Enfin nous transformons les tokens en tensor pour les passer dans le fameux transformer. Seule la dernière couche est conservée pour faire la classification.

In [25]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)


#### Entraîner un modèle logistique

Comme nous avons seulement besoin du premier token (CLS qui signifie classification) pour le modèle logistique, nous faisons en slice avec [:,0,:].

In [26]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels = commentaires_raw['label']
labels

Nous commençons par faire un split train/test avec Scikit-Learn. Ensuite nous utilisons Grid Search pour essayer de trouver le meilleur paramètre. Finalement on entraîne le modèle.

In [28]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [29]:
## grid search
# Grid search
from sklearn.svm import SVC
#parameters = {'C': np.linspace(0.00, 100, 20)}
parameters_svc = {'kernel':('linear', 'rbf'), 'C':np.linspace(0.0015, 100, 20)}
grid_search = GridSearchCV(SVC(), parameters_svc)
#grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 15.790736842105265, 'kernel': 'rbf'}
best scrores:  0.7226666666666667


In [30]:
lr_clf = LogisticRegression(C=grid_search.best_params_['C'])
lr_clf.fit(train_features, train_labels)
y_pred = lr_clf.predict(test_features)


Nous arrivons donc à un score de 72%, qui est un grand mais toujours faible.. Cela peut s'expliquer pas les mauvaises par la notation (note attribuée) humaine qui peut ne pas très fiable au caracteres positif ou négatif du commentaire.

In [ ]:
lr_clf.score(test_features, test_labels)


In [32]:
print(accuracy_score(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))
print(classification_report(test_labels, y_pred))

0.68
[[46 18]
 [22 39]]
              precision    recall  f1-score   support

           0       0.68      0.72      0.70        64
           1       0.68      0.64      0.66        61

    accuracy                           0.68       125
   macro avg       0.68      0.68      0.68       125
weighted avg       0.68      0.68      0.68       125



## CONCLUSION PARTIELLE

#### Nous avons tenter des classifier des commentaires des appréciations des produits sur Amazons.. 
#### Nous notons que les classifieurs utilisés donnent des resulalts mitigés voir pas du tout bon. 
#### Ce pendant la puissance de ses algorithmes sur les méthodes de classificatios prouvées, nous laisse croire que les notations (attribution des étoiles)des commentaires faits par l'humain ne sont pas toujours pas conforme avec le commentaire.

In [ ]:
accuracy_score(y_test_wv, y_pred)